In [1]:
from ampel.contrib.hu.examples.t2.T2ExamplePolyFit import T2ExamplePolyFit
from ampel.pipeline.logging.LoggingUtils import LoggingUtils

Pickle load a lightcurve object (a lightcurve instance be provided as input of any T2Runnable unit)

In [2]:
import pickle
lc = pickle.load(open("../src/ampel/contrib/hu/examples/t2/lightcurve.pickle", "rb"))

In [3]:
lc.get_values("mag")

[23.856793629352122,
 23.00805766234931,
 21.595161397095065,
 23.447385814760864,
 23.064335890370327,
 21.117456462709832,
 21.69892544131502,
 20.54556957554393,
 20.881180998046148,
 20.340208010886663,
 20.187509359597723,
 20.40742267815947,
 20.588474220085388,
 20.556809476480602]

Instanciate example t2 class

In [4]:
t2_runnable = T2ExamplePolyFit(  
    LoggingUtils.get_logger(),
    base_config={"fitFunction": "polyfit"}
)

# Python implementation

A T2 class *must* inherit ```ampel.abstract.AbsT2Unit``` and implement the two functions (otherwise exception will be throwed):
- ```__init__(self, logger, base_config)```

```
'logger': instance of logging.Logger (std python module 'logging')
	-> example usage: logger.info("this is a log message")

'base_config': optional dict loaded from ampel config section: 
	-> t2_units->POLYFIT->baseConfig
```

- ```run(self, light_curve, run_config)```

```
'light_curve': instance of ampel.base.LightCurve. See LightCurve docstring for more info.

'run_config': dict instance containing run parameters defined in ampel config section:
	t2_run_config->POLYFIT_[run_config_id]->runConfig 
	whereby the run_config_id value is defined in the associated t2 document.
	In the case of POLYFIT, run_config_id would be either 'default' or 'advanced'.
	A given channel (say HU_SN_IA) could use the runConfig 'default' whereas 
	another channel (say OKC_SNIIP) could use the runConfig 'advanced'

run() must return either:
	* A dict instance containing the values to be saved into the DB
		-> IMPORTANT: the dict *must* be BSON serializable, that is:
			import bson
			bson.BSON.encode(<dict instance to be returned>)
		must not throw a InvalidDocument Exception
	* One of these T2RunStates flag member:
		MISSING_INFO:  reserved for a future ampel extension where 
					   T2s results could depend on each other
		BAD_CONFIG:	   Typically when run_config is not set properly
		ERROR:		   Generic error
		EXCEPTION:     An exception occured


```

The following method is called automatically by the Ampel T2 controller after the t2runnable class instanciation (based on the ampel config)

In [5]:
t2_runnable.set_base_parameters(
    {"fit_function": "polyfit"}
)

The Ampel T2 controller will call the method ```run``` and provide an instance of ampel.base.LightCurve and the dict instance  ```run_config```

In [5]:
t2_runnable.run(
    light_curve=lc, 
    run_config={'degree': 3}
)

2018-03-08 13:26:15 T2ExamplePolyFit.py:125 run() INFO Please use 'self.logger' for logging
2018-03-08 13:26:15 T2ExamplePolyFit.py:126 run() DEBUG By doing so, log entries will be automatically recorded into the database


{'chi2': 4.276766382894305,
 'polyfit': [0.00010168208142339463,
  -17.710519722219509,
  1028245.6388793393,
  -19899452971.043541]}

T2Controller will automatically save the dict returned by ```run``` into the Ampel database<br>
Please note that T2Controller will also complete the dictionary with the following entries:
- ```version```: the version number returned by get_version() of the T2ExamplePolyFit instance
- ```run_config```: the run config parameters provided to the method ```run``` (originating from the value of ```runConfig``` contained in the corresponding T2 database document)


----------
Note: the method ```run``` must return a BSON encodable dictionnary.<br>
If you look at the source code of ```run```, you'll see that the  numpy array outputed by polyfit is casted into a python list. This is because a numpy array is not BSON encodable:<br><font color="red"> (following error and traceback are created on purpose)</font>

In [6]:
import bson, numpy
bson.BSON.encode({'a': numpy.array([1,2,3])})

InvalidDocument: Cannot encode object: array([1, 2, 3])

whereas a python list is:

In [7]:
bson.BSON.encode({'a': [1,2,3]})

b'"\x00\x00\x00\x04a\x00\x1a\x00\x00\x00\x100\x00\x01\x00\x00\x00\x101\x00\x02\x00\x00\x00\x102\x00\x03\x00\x00\x00\x00\x00'